In [1]:
# IMPORT LIBRARIES
import psycopg2 as pg2
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import snowflake.connector

In [2]:
#CONNECT TO SNOWFLAKE WITH REPORTER CREDENTIALS
conn = snowflake.connector.connect(
    user='emirkabasoglu', #emirkabasoglu
    password='7Bb13287', #7Bb13287
    account='se63477.north-europe.azure',
    warehouse='TRANSFORMING_WH',
    database='PROD_ANALYTICS_DB'
    )

In [19]:
#START THE CURSOR AND RUN THE QUERY
cur = conn.cursor()
cur.execute(''' 

--BOARD METRICS NETSUITE 



select
t1.econ_id as id,
t1.customer_id,
t1.company_name,
t2.item_id,
t2.item,
t2.product_type,
t2.quantity,
t2.value_arr_eur as value,
t2.commit_date as commit,
t2.start_date as "start",
t2.end_date as end ,
t2.cancelled,
t1.region,
case when t3.fte_size < 250 then 'smb' 
     when (t3.fte_size > 249 and 5001 > t3.fte_size)  then 'mm'
     when t3.fte_size is null then null
     else 'ent' end as size,
case when t2.product_type ilike ('%content%') then 'content' 
     else 'software' end as product,
case when t2.product_type ilike ('%infinite%') then 'infinite'
     when t2.product_type ilike ('%content%')  then ''
     else 'people' end as type,
case when t6.inbound_lead_source in ('ppc - google') then 'google'
     when t6.inbound_lead_source in ('ppc - bing') then 'bing' 
     when t6.inbound_lead_source in ('ppc - facebook') then 'facebook'
     when t6.inbound_lead_source in ('ppc - linkedin') then 'linkedin'       
     when t6.inbound_lead_source in ('review db - capterra','review db - software advice','review db - getapp', 'review db - elearning industry', 'review db - people managing people', 'review db - select software', 'review db - softwaresuggest') then 'review' 
     when t6.inbound_lead_source in ('referral - other') then 'other'
     else null end as inbound_source
from prod_analytics_db.dbt_deploy.netsuite_customers t1
left join prod_analytics_db.dbt_deploy.netsuite_subscriptions t2 
on t1.customer_id=t2.customer_id 
left join prod_analytics_db.dbt_deploy.int_customer_firmographic_assigned t3 on t3.company_id=t1.hubspot_id
left join prod_analytics_db.dbt_deploy.stg_hubspot__deal t6 
on try_cast(t6.deal_id as number) = try_cast(t2.hubspot_deal_id as number)
where  t2.item_id is not null

''')


In [20]:
# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)

In [21]:
# ASSIGN TO MAIN DF FOR FUTHER TRANSFORMATION
df_main = df_items_db_fetched

In [22]:
df_main.columns

Index(['ID', 'CUSTOMER_ID', 'COMPANY_NAME', 'ITEM_ID', 'ITEM', 'PRODUCT_TYPE',
       'QUANTITY', 'VALUE', 'COMMIT', 'start', 'END', 'CANCELLED', 'REGION',
       'SIZE', 'PRODUCT', 'TYPE', 'INBOUND_SOURCE'],
      dtype='object')

In [23]:
#CONVERT ALL THE COLUMN NAMES LOWER CASE
df_main.columns= df_main.columns.str.lower()
##df_main['size'] = df_main['size'].str.lower()
df_main['region'] = df_main['region'].str.lower()



In [24]:
df_main.head(5)

,id,customer_id,company_name,item_id,item,product_type,quantity,value,commit,start,end,cancelled,region,size,product,type,inbound_source
0,400001,2855,Nuuday A/S,420,People Service Plan Lms Plus,People Subscription,1.0,1072.258,2020-11-30,2021-01-01,2022-12-31,y,norben,mm,software,people,None
1,400001,2855,Nuuday A/S,412,People Performance Management,People Subscription,250.0,10568.250,2020-11-30,2021-01-01,2023-12-31,y,norben,mm,software,people,None
2,400001,2855,Nuuday A/S,410,People Learning Management,People Subscription,250.0,8723.000,2020-11-30,2021-01-01,2023-12-31,y,norben,mm,software,people,None
3,400001,2855,Nuuday A/S,420,People Service Plan Lms Plus,People Subscription,1.0,1104.466,2020-11-30,2023-01-01,2023-12-31,y,norben,mm,software,people,None
4,400001,2855,Nuuday A/S,320,Content Grape Full Library,Content - External,1.0,2684.000,2020-11-30,2022-06-01,2022-12-31,y,norben,mm,content,,None


In [25]:
# OUTPUT FINAL FILE WITH ALL DIFFERENCES
df_main.to_excel('Board_Metrics_Input_File_Netsuite.xlsx')